In [ ]:
# import the usual resources
import matplotlib.pyplot as plt
import numpy as np

# import utilities to keep workspaces alive during model training
from workspace_utils import active_session

# watch for any changes in model.py, if it changes, re-load it automatically
%load_ext autoreload
%autoreload 2

## TODO: Define the Net in models.py

import torch
import torch.nn as nn
import torch.nn.functional as F


from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# the dataset we created in Notebook 1 is copied in the helper file `data_load.py`
from data_load import FacialKeypointsDataset
# the transforms we defined in Notebook 1 are in the helper file `data_load.py`
from data_load import Rescale, RandomCrop, Normalize, ToTensor, RandomMirror

In [ ]:
import glob
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.image as mpimg
import pandas as pd
import cv2

In [ ]:
# read data 
csv_file='/data/training_frames_keypoints.csv'
root_dir='/data/training/'
idx = 7
key_pts_frame = pd.read_csv(csv_file)
image_name = os.path.join(root_dir, key_pts_frame.iloc[idx, 0])

image = mpimg.imread(image_name)
if(image.shape[2] == 4):
    image = image[:,:,0:3]
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
key_pts = key_pts_frame.iloc[idx, 1:].as_matrix()
key_pts = key_pts.astype('float').reshape(-1, 2)

print(image.shape)
print(key_pts.shape)
plt.figure()
plt.imshow(image, cmap='gray')
plt.scatter(key_pts[:, 0], key_pts[:, 1], s=20, marker='.', c='m')

In [ ]:
### test the mirror function 
def mirror(image, key_pts):
    # image, key_pts = sample['image'], sample['keypoints']
    h, w = image.shape[:2]
    img = image.copy()
    print('h: {}, w: {}'.format(h, w))
    print(img.shape)
    img = np.flip(img, axis=1)
    print(key_pts[:5,:])
    key_pts[:,0] = w - key_pts[:,0]
    print(key_pts[:5,:])
    return img, key_pts

def rescale(image, key_pts, out_size):
    h, w = image.shape[:2]
    img = image.copy()
    if h > w:
        new_h, new_w = out_size * h / w, out_size
    else:
        new_h, new_w = out_size, out_size * w / h
    new_h, new_w = int(new_h), int(new_w)
    img = cv2.resize(img, (new_w, new_h))
    # scale the pts, too
    key_pts = key_pts * [new_w / w, new_h / h]
    return img, key_pts

In [ ]:
def preprocess(image, kpts, outsize=96):
    img, new_kpts = rescale(image, kpts, outsize)
    img, new_kpts = mirror(img, new_kpts)
    return img, new_kpts

m_img, m_kpts = preprocess(image, key_pts)
plt.figure()
plt.imshow(m_img, cmap='gray')       
plt.scatter(m_kpts[:, 0], m_kpts[:, 1], s=20, marker='.', c='g')